In [1]:
import sys
import os
import pandas as pd

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-05-11 09:32:51.787774


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("6H")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-05-11T06:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-05-11 06:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-04-12 06:00:00+0000', tz='UTC')

In [24]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 3 months ago (i.e., 12 weeks).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 12 weeks (nearly 3 months)
    historical_from_date = from_date - timedelta(weeks=12)
    historical_to_date = to_date - timedelta(weeks=12)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.started_at >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.started_at < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 12 weeks to simulate recent data
    rides['started_at'] += timedelta(weeks=12)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'started_at'], inplace=True)

    return rides

In [25]:
fetch_data_from = pd.to_datetime(fetch_data_from).tz_localize(None)
fetch_data_to = pd.to_datetime(fetch_data_to).tz_localize(None)

In [26]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2025-01.
Loading data for 2025-01...
Total records: 2,119,110
Valid records: 69,006
Records dropped: 2,050,104 (96.74%)
Successfully processed data for 2025-01.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2025-02.
Loading data for 2025-02...
Total records: 2,025,432
Valid records: 67,297
Records dropped: 1,958,135 (96.68%)
Successfully processed data for 2025-02.
Combining all monthly data...
Data loading and processing complete!


In [27]:
rides

,started_at,pickup_location_id
8266,2025-04-12 06:02:56.844,5626
23104,2025-04-12 07:08:12.700,5626
12192,2025-04-12 07:13:19.765,5626
15573,2025-04-12 07:14:21.812,5626
14065,2025-04-12 07:44:32.899,5626
...,...,...
66889,2025-05-11 00:01:20.582,6072
66248,2025-05-11 00:27:49.903,6072
65866,2025-05-11 01:12:53.787,6072
66731,2025-05-11 01:43:14.463,6072


In [28]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [29]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-04-12 06:00:00,5626,130
1,2025-04-12 12:00:00,5626,212
2,2025-04-12 18:00:00,5626,119
3,2025-04-13 00:00:00,5626,48
4,2025-04-13 06:00:00,5626,142
...,...,...,...
575,2025-05-10 00:00:00,6072,8
576,2025-05-10 06:00:00,6072,74
577,2025-05-10 12:00:00,6072,98
578,2025-05-10 18:00:00,6072,26


In [30]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         580 non-null    datetime64[ns]
 1   pickup_location_id  580 non-null    int16         
 2   rides               580 non-null    int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 6.9 KB


In [31]:
import hopsworks


import hopsworks 
project = hopsworks.login()
feature_store = project.get_feature_store(name='thanoojl_featurestore')
feature_group = feature_store.get_feature_group('time_series_6h_feature_group', version=1)

2025-05-11 05:51:47,620 INFO: Initializing external client
2025-05-11 05:51:47,622 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-11 05:51:49,539 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [32]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 580/580 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: time_series_6h_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214697/jobs/named/time_series_6h_feature_group_1_offline_fg_materialization/executions


(Job('time_series_6h_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)